In [1]:
!pip install pandas numpy jieba tqdm jupyter

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
# PyTorch for Windows, CPU version
!pip3 install torch torchvision torchaudio

In [ ]:
# Please refer to https://pytorch.org to get more installation options, including cuda (GPU).

In [1]:
import copy
import math
import pandas as pd
import numpy as np
import jieba
# import nltk
# import spacy
import json
from collections import Counter
from tqdm.notebook import tqdm
import torch
from torch.nn import functional as F

In [2]:
datapath = 'data/test-f.csv'

In [4]:
data = pd.read_csv(datapath,index_col=0)
data_records = data.to_dict('records')

In [14]:
data_records[0].keys()

dict_keys(['id', 'content', 'task-1', 'task-2'])

In [5]:
sample_string = data_records[42]['content']

In [9]:
print(sample_string)
list(jieba.cut(sample_string))

有些事越想要越得不到，有些梦只能相信，是这样吗


['有些', '事越', '想要', '越', '得不到', '，', '有些', '梦', '只能', '相信', '，', '是', '这样', '吗']

In [10]:
def cut_sentence(sent): # tokenizer
    """
    :param sent: str
    :return:  list

    Write your own code here
    """

    return jieba.cut(sent)

In [16]:
for idx,r in enumerate(data_records):
    data_records[idx]['tokens_1'] = list(cut_sentence(r['content']))
data_records[0]

{'id': 1,
 'content': '宿舍要民汉合宿了为毛都大三了还要折腾我',
 'task-1': 'neg',
 'task-2': 'angry',
 'tokens_1': ['宿舍', '要民汉合宿', '了', '为', '毛', '都', '大三', '了', '还要', '折腾', '我']}

In [12]:
# try your function
cut_sentence(sample_string)

<generator object Tokenizer.cut at 0x16d494880>

In [7]:
def get_stopwords():
    """
    You can provide A better list, here is an example.
    https://github.com/goto456/stopwords
    """
    return
stop_words = get_stopwords()

In [8]:
def strip_stopwords(words,stopwords):
    """
    :param words: LIST
    :param stopwords: LIST
    :return: LIST OF WORDS AFTER FILTERING
    """
    return

In [9]:
strip_stopwords(cut_sentence(sample_string),stop_words)

['事越', '想要', '越', '得不到', '梦', '只能', '相信']

In [10]:
"""
Now loop over all sentences to get their words,
then save the results to the variable data_records for later call.
"""
def process_content(data_records):
    """
    Your code here....
    :return: list
    """

    return

recordsWithWords = process_content(data_records)

In [11]:
wordCorpus = []
for item in recordsWithWords:
    wordCorpus+=item['words']
print(wordCorpus[:20])

['宿舍', '要民汉合宿', '毛', '大三', '折腾', '早上', '竟然', '变成', '一个', '无理取闹', '…', '…', '多年', '周天', '先生', '率', '智多星', '律师', '策划师', '团队']


In [12]:
# construct the vocabulary list

def construct_vocab(wordCorpus,save=True):
    """
    :param word_list:list
    :return: a vocab set
    [Attention]: A set means there is no repeat item.
    """

    #Your code here..

    ## You don't have to modify the following code
    if save:
        res = vocab
        res.sort()
        json.dump(res,open('results/vocab.json','w'),ensure_ascii=False)
    return vocab
vocab = construct_vocab(wordCorpus)
word2id = {w:idx for idx,w in enumerate(vocab)}

In [13]:
def wordFreq(wordList):
    return

In [14]:
vocabFreq = { w:0 for idx,w in enumerate(vocab)}
wordCounted = wordFreq(wordCorpus)
vocabFreq.update(wordCounted)

In [15]:
def get_total_freq_of_sentence(wordList,Freq):
    return

def convert_words2freqs(wordList,Freq):
    return

TF Calculation

为每个句子中的单词计算对应的 在句子中的 词频 aka TF: term frequency

 $tf_{i,j} = \frac{n_{i,j}}{\sum_{k}n_{k,j}}$,

i 是词语索引, j 是句子索引


 each sentence will construct its features on the scale of the whole vocabulary list

In [16]:
def get_tf_feat(wordList,vocab):

    return

In [17]:
vocab_size = len(vocab)
len(vocab)

21624

In [18]:
recordsWithTF = copy.deepcopy(recordsWithWords)
for idx,item in enumerate(tqdm(recordsWithTF)):
    tf = get_tf_feat(item['words'],vocab)
    item['TF_array'] = tf

  0%|          | 0/5000 [00:00<?, ?it/s]

In [19]:
[item for item in recordsWithTF[42]['TF_array'].tolist() if item!=0]

[0.14285714285714285,
 0.14285714285714285,
 0.14285714285714285,
 0.14285714285714285,
 0.14285714285714285,
 0.14285714285714285,
 0.14285714285714285]

IDF Calculation

$idf_i = lg\frac{|D|}{1 + |{j:t_i \in d_j}|}$

|D|: the total of sentences
$d_j$: the specific j-th sentence
|{j:t_i \in d_j}|: the number of sentences that contains the word $t_i$

1+: avoid zero division when the word not in any sentences

In [20]:
def calculateCorpusIDF(vocab,ListOfWordList):

    return


def get_idf_feat(wordList,idf_values):
    assert idf_values != None

    return

In [21]:
idf_values = calculateCorpusIDF(vocab,[item['words'] for item in data_records])

  0%|          | 0/21624 [00:00<?, ?it/s]

In [22]:
recordsWithTFIDF = copy.deepcopy(recordsWithTF)
for idx,r in enumerate(recordsWithTFIDF):
    r['IDF'] = get_idf_feat(r['words'],idf_values)
    recordsWithTFIDF[idx] = r

In [23]:
recordsWithTFIDF[42]

{'id': 43,
 'content': '有些事越想要越得不到，有些梦只能相信，是这样吗',
 'task-1': 'neg',
 'task-2': 'sad',
 'words': ['事越', '想要', '越', '得不到', '梦', '只能', '相信'],
 'TF_array': array([0., 0., 0., ..., 0., 0., 0.]),
 'IDF': array([0., 0., 0., ..., 0., 0., 0.])}

In [24]:
[item for item in recordsWithTFIDF[42]['IDF'].tolist() if item!=0]

[7.824046010856292,
 4.688551794927142,
 6.725433722188183,
 4.645992180508347,
 4.990832666800076,
 5.221356325411908,
 5.115995809754082]

In [25]:
def get_tf_idf_feat(wordList,idf_values,vocab):

    return

In [26]:
recordsWithTFIDF = copy.deepcopy(recordsWithTFIDF)

for idx,r in enumerate(tqdm(recordsWithTFIDF)):
    r['TFIDF'] = get_tf_idf_feat(r['words'],idf_values,vocab)
    recordsWithTFIDF[idx] = r

  0%|          | 0/5000 [00:00<?, ?it/s]

In [27]:
recordsWithTFIDF[42]

{'id': 43,
 'content': '有些事越想要越得不到，有些梦只能相信，是这样吗',
 'task-1': 'neg',
 'task-2': 'sad',
 'words': ['事越', '想要', '越', '得不到', '梦', '只能', '相信'],
 'TF_array': array([0., 0., 0., ..., 0., 0., 0.]),
 'IDF': array([0., 0., 0., ..., 0., 0., 0.]),
 'TFIDF': array([0., 0., 0., ..., 0., 0., 0.])}

In [28]:
[item for item in recordsWithTFIDF[42]['TFIDF'].tolist() if item!=0]

[1.117720858693756,
 0.6697931135610203,
 0.9607762460268832,
 0.6637131686440495,
 0.7129760952571537,
 0.7459080464874154,
 0.7308565442505831]

相似度计算

In [29]:
def similarity(vec_a,vec_b):
    """
    :param vec_a: np array
    :param vec_b: np array
    :return: float
    """
    return

In [36]:
tfidfs = np.array([item['TFIDF'] for item in recordsWithTFIDF])

In [31]:
sim = np.zeros((len(tfidfs),len(tfidfs)))
for x in tqdm(range(len(tfidfs))):
    for y in range(len(tfidfs)):
        sim[x,y] = similarity(tfidfs[x],tfidfs[y])

  0%|          | 0/5000 [00:00<?, ?it/s]


KeyboardInterrupt

